Yale Hackathon

Task 2A

In [ ]:
from functions_lib import *
import dynamiqs as dq
from scipy.integrate import dblquad
from scipy.ndimage import gaussian_filter
from scipy.interpolate import RegularGridInterpolator
import numpy as np


Input noisy wigner function (real-world Wigner function measurements suffer from affine distortions)

In [ ]:
# input is noisy wigner function
wig_file_path = "/common/home/sps247/code/yq25_alice-bob_challenge_braket_bros/data/synthetic/noisy_wigner_0.pickle"
wig_noisy = load_data_from_pickle(wig_file_path)
p_file_path = "cat_rho.pickle" # path to pickle file containing new rho, output from 1b
p_new = load_data_from_pickle(p_file_path)

Using the output of the new density matrix from Task 1b, find what the true Wigner function should be

In [ ]:
# outputs the perfect wigner function
wig_perf = dq.wigner(state=p_new)

Use least squares regression analysis to estimate the value of a to fit the model to the data as best as possible

In [ ]:
def estimate_a(wig_noisy, wig_perf):
    num = np.sum(wig_noisy * wig_perf)
    denom = np.sum(wig_perf ** 2)
    a = num / denom

Knowing that the edges of the phase space of the Wigner function should approach 0, we can estimate the value of b. Then, we use Monte Carlo to minimize the value of b.

In [ ]:
def estimate_b(wig_noisy, wig_perfect, a, n_simulations=1000):
    # estimate b using edges of phase space
    top_edge = wig_noisy[0, :]
    bottom_edge = wig_noisy[-1, :]
    left_edge = wig_noisy[:, 0]
    right_edge = wig_noisy[:, -1]

    # combine edge values
    edges = np.concatenate((top_edge, bottom_edge, left_edge, right_edge))

    # estimate b as mean of edge values
    b_initial = np.mean(edges)

    # want to minimize b
    b_range = 0.1 # arbitrarily set, adjust as needed
    b_min = b_initial - b_range
    b_max = b_initial + b_range

    # use monte carlo simulation to minimize value of b
    smallest_b = None
    min_noise = float('inf') # want to minimize noise, so initialize to infinity

    for i in range(n_simulations):
        b = np.random.uniform(b_min, b_max)
        wig_simulated = a * wig_perf + b # simulate noisy wigner function based on random b value
        noise = np.mean((wig_simulated - wig_noisy) ** 2) # calculate noise as mean squared error

        # update smallest b if noise is smaller than current minimum
        if noise < min_noise:
            min_noise = noise
            smallest_b = b


After finding the values of a and b, we apply to normalization constraint to correct the affine distortion. Since the Wigner function is represented as a 2D discrete array, we interpolate it so that we can find its integral to confirm that the function satisfies the normalization constraint:
$$
\int \int W(x,p) \, dx \, dp = 1
$$
Then, we apply the correction which is:
$$
W_{\text{corrected}}(x, p) = \frac{1}{a} \cdot \left(W_{\text{measured}}(x, p) - b\right)
$$


In [ ]:
def correct_affine_distortion(wig_noisy, x, p):
    # create interpolator to check integral value
    x = np.linspace(wig_noisy[:, -1], lambda x: wig_noisy[0, :], wig_noisy.shape[0])
    p = np.linspace(wig_noisy[:, 0], wig_noisy[:, -1], wig_noisy.shape[1])
    interpolator = RegularGridInterpolator((x, p), wig_noisy)

    def interpolate(x, p):
        return interpolator((x, p))
    
    integral, error = dblquad(interpolate, wig_noisy[:, 0], wig_noisy[:, -1], lambda x: wig_noisy[-1, :], lambda x: wig_noisy[0, :])

    # check normalization
    if not np.isclose(integral, 1):
        raise ValueError("Wigner function is not normalized.")
    
    else:
        # correct wigner function
        wig_corrected = (1 / a) * (wig_noisy - b)

    

Apply a 2D Gaussian filter to reduce noise in the Wigner distribution. We choose to set $\sigma$ as 0.1.

In [ ]:
def gauss_filter(wig_corrected, sigma):
    wig_smoothed = gaussian_filter(wig_corrected, sigma=0.1) # sigma arbitrarily set, may be changed
    return wig_smoothed